# Transporte aéreo - parte b

In [7]:
import gurobipy as gp
from gurobipy import GRB

In [8]:
tipos = 3
rotas = 4
cap = [40, 60, 100]
frota = [7, 8, 6]
#demanda = [650, 710, 610, 950]
demanda = [990, 1080, 930, 1440]
numViagens = [[3, 2, 2, 1], [4, 3, 3, 2], [5, 4, 4, 2]]
custo = [[1500, 1900, 2100, 2800],[2100, 2600, 2800, 3700],[3200, 3700, 3900, 5800]]
penalidade = [2500, 3000, 3500, 6000]

In [13]:
# Modelo
m = gp.Model("Aereo")

# Variáveis de decisão
x = []
for i in range(tipos):
    x.append([])
    for j in range(rotas):
        x[i].append(m.addVar(vtype=GRB.INTEGER, obj=0.0, name =f"x_{i}_{j}"))

y = []
for i in range(tipos):
    y.append([])
    for j in range(rotas):
        y[i].append(m.addVar(vtype=GRB.INTEGER, name =f"y_{i}_{j}"))

z = []
for j in range(rotas):
    z.append(m.addVar(vtype=GRB.INTEGER, name =f"z_{j}"))

# Restrições
# Não usar mais aeronaves do que o disponível
for i in range(tipos):    
    expr = 0
    for j in range(rotas):
        expr += x[i][j]
    m.addConstr(expr <= frota[i], f"c01_{i}")

# Atender à demanda de cada rota
for j in range(rotas):
    expr = z[j]
    for i in range(tipos):    
        expr += y[i][j]*cap[i]
    m.addConstr(expr >= demanda[j], f"c02_{j}")

# Restrição de vínculo
for i in range(tipos):    
    for j in range(rotas):
        expr = y[i][j]-numViagens[i][j]*x[i][j]
        m.addConstr(expr <= 0, f"c03_{i}_{j}")

# Função objetivo
expr = 0
for i in range(tipos):    
    for j in range(rotas):
        expr += custo[i][j]*y[i][j]
for j in range(rotas):
    expr += penalidade[j]*z[j]*0

m.setObjective(expr, GRB.MINIMIZE)
m.update()
m.write("Modelo_TransporteAereo_b.lp")


In [14]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 19 rows, 28 columns and 52 nonzeros
Model fingerprint: 0xf1e0b77d
Variable types: 0 continuous, 28 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+03, 6e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 1e+03]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [11]:
for j in range(rotas):
    print(f"Rota {j} -> Demanda Não Atendida: {z[j].x}")


Rota 0 -> Demanda Não Atendida: 150.0
Rota 1 -> Demanda Não Atendida: 280.0
Rota 2 -> Demanda Não Atendida: 130.0
Rota 3 -> Demanda Não Atendida: 80.0


In [12]:
for i in range(tipos):
    for j in range(rotas):
        print(f"Tipo {i} Rota {j} -> Quant. Alocada: {x[i][j].x}, Núm. Viagens: {y[i][j].x}")


Tipo 0 Rota 0 -> Quant. Alocada: 7.0, Núm. Viagens: 21.0
Tipo 0 Rota 1 -> Quant. Alocada: -0.0, Núm. Viagens: 0.0
Tipo 0 Rota 2 -> Quant. Alocada: -0.0, Núm. Viagens: 0.0
Tipo 0 Rota 3 -> Quant. Alocada: -0.0, Núm. Viagens: -0.0
Tipo 1 Rota 0 -> Quant. Alocada: 0.0, Núm. Viagens: -0.0
Tipo 1 Rota 1 -> Quant. Alocada: -0.0, Núm. Viagens: 0.0
Tipo 1 Rota 2 -> Quant. Alocada: 0.0, Núm. Viagens: 0.0
Tipo 1 Rota 3 -> Quant. Alocada: 8.0, Núm. Viagens: 16.0
Tipo 2 Rota 0 -> Quant. Alocada: 0.0, Núm. Viagens: 0.0
Tipo 2 Rota 1 -> Quant. Alocada: 2.0, Núm. Viagens: 8.0
Tipo 2 Rota 2 -> Quant. Alocada: 2.0, Núm. Viagens: 8.0
Tipo 2 Rota 3 -> Quant. Alocada: 2.0, Núm. Viagens: 4.0
